In [2]:
### preparing model for extracted features from the payloads

import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# appending path for importing common process ua

import sys 
sys.path.append('/Users/kisho/Documents/GitHub/Risk_Quantification')
sys.path
from WAF.utils.util import process_ua


payloads_with_feature_df = pd.read_csv("/Users/kisho/Documents/GitHub/Risk_Quantification/WAF_Model_Trainer/data_collection/combined_payloads_with_features.csv" , index_col=0)
payloads_df = pd.read_csv("/Users/kisho/Documents/GitHub/Risk_Quantification/WAF_Model_Trainer/data_collection/combined_payloads.csv" , index_col=0)


In [3]:
#dropping payload column

payloads_with_feature_df.drop(["payloads"],axis=1,inplace=True)

payloads_with_feature_df

,length,nonprintable_keywords,special_chars,punctuation_chars,js_events,html_tags,sql_keywords,percentage_count,spaces_count,is_malicious
0,5,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
41575,58,0,9,9,0,1,0,0,1,1
41576,49,0,13,13,0,2,0,0,1,1
41577,73,0,11,11,0,2,0,0,1,1
41578,114,0,36,36,0,8,0,25,1,1


In [3]:
def dump(model , filename):
    with open(filename, "wb") as f:
        joblib.dump(model , f)

x = payloads_with_feature_df.drop(["is_malicious"],axis=1).values 
y = payloads_with_feature_df["is_malicious"].values



#Dont Run take long time to run

In [13]:
x = payloads_with_feature_df.drop(["is_malicious"],axis=1).values 
y = payloads_with_feature_df["is_malicious"].values
x_train , x_test , y_train , y_test = train_test_split(x,y,random_state=1 , test_size=0.2)

from sklearn.metrics import classification_report

[[  5   0   0 ...   0   0   0]
 [  4   0   0 ...   0   0   0]
 [  3   0   0 ...   0   0   0]
 ...
 [ 73   0  11 ...   0   0   1]
 [114   0  36 ...   0  25   1]
 [302   0 122 ...   0   0   2]]


In [20]:
#Custom Feature Results

clf1 = RandomForestClassifier(n_estimators=30, criterion="entropy", max_depth=30)
clf2 = DecisionTreeClassifier(max_depth=30, criterion="gini")
clf3 = KNeighborsClassifier(n_neighbors=10)

ens_model = VotingClassifier(estimators=[('clf1', clf1), ('clf2', clf2), ('clf3', clf3)], voting='soft')
ens_model.fit(x_train,y_train)
file_path = "C:/Users/kisho/Documents/GitHub/Risk_Quantification/WAF_Models/Payload_Models"
dump(ens_model , filename=f"{file_path}/ens_custom_feature.pkl")
y_pred = ens_model.predict(x_test)

print("[+] ACCURACY \n")

print("Ensembled Classifier : " , accuracy_score(y_test , y_pred))

print(classification_report(y_test , y_pred))

[array([1., 1., 1., ..., 0., 1., 1.]), array([1., 1., 1., ..., 0., 1., 1.]), array([1., 1., 1., ..., 0., 1., 1.])]
-500399958597220.0


In [18]:
### real world payloads feature extraction

# from WAF_Model_Trainer.utils import PayloadSpecialFeatureExtractor
# import pandas as pd


# def extract_feature(payload : str) -> pd.DataFrame:
    
#     d = pd.DataFrame([payload] , columns=["payloads"])

#     o  = PayloadSpecialFeatureExtractor(d)
#     o.fit()

#     return d 

# payload = "?id=1 AND SELECT SUBSTR(table_name,1,1) FROM information_schema.tables = 'A'"


# d = extract_feature(payload)
# print(d)

# ens_model.predict(d.drop(["payloads"],axis=1).values)

                                            payloads  length  \
0  ?id=1 AND SELECT SUBSTR(table_name,1,1) FROM i...      76   

   nonprintable_keywords  special_chars  punctuation_chars  js_events  \
0                      0             12                 12          0   

   html_tags  sql_keywords  percentage_count  spaces_count  
0          0             6                 0             7  


array([1], dtype=int64)